# 图像卷积

互相关运算

In [5]:
import torch
from torch import nn
from d2l import torch as d2l

def corr2d(X, K):  
    """计算二维互相关运算"""
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

验证上述二维互相关运算的输出

In [2]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

实现二维卷积层

In [3]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

卷积层的一个简单应用：
检测图像中不同颜色的边缘

In [4]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [5]:
K = torch.tensor([[1.0, -1.0]])

输出`Y`中的1代表从白色到黑色的边缘，-1代表从黑色到白色的边缘

In [6]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

卷积核`K`只可以检测垂直边缘

In [7]:
corr2d(X.t(), K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

学习由`X`生成`Y`的卷积核

In [8]:
conv2d = nn.Conv2d(1,1, kernel_size=(1, 2), bias=False)

X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'epoch {i+1}, loss {l.sum():.3f}')

epoch 2, loss 10.021
epoch 4, loss 1.974
epoch 6, loss 0.451
epoch 8, loss 0.125
epoch 10, loss 0.041


所学的卷积核的权重张量

In [9]:
conv2d.weight.data.reshape((1, 2))

tensor([[ 1.0036, -0.9655]])

In [11]:
conv2d.weight.data.shape

torch.Size([1, 1, 1, 2])

## 练习

1.构建一个具有对角线边缘的图像X。
- 如果将本节中举例的卷积核K应用于X，会发生什么情况？
- 如果转置X会发生什么？
- 如果转置K会发生什么？

In [14]:
X = torch.zeros((6,8))
for i in range(6):
  X[i, i] = 1
  X[i, i+1] = 1
  X[i, i+2] = 1
X

tensor([[1., 1., 1., 0., 0., 0., 0., 0.],
        [0., 1., 1., 1., 0., 0., 0., 0.],
        [0., 0., 1., 1., 1., 0., 0., 0.],
        [0., 0., 0., 1., 1., 1., 0., 0.],
        [0., 0., 0., 0., 1., 1., 1., 0.],
        [0., 0., 0., 0., 0., 1., 1., 1.]])

In [15]:
corr2d(X, K)

tensor([[ 0.,  0.,  1.,  0.,  0.,  0.,  0.],
        [-1.,  0.,  0.,  1.,  0.,  0.,  0.],
        [ 0., -1.,  0.,  0.,  1.,  0.,  0.],
        [ 0.,  0., -1.,  0.,  0.,  1.,  0.],
        [ 0.,  0.,  0., -1.,  0.,  0.,  1.],
        [ 0.,  0.,  0.,  0., -1.,  0.,  0.]])

In [16]:
corr2d(X.t(), K)

tensor([[ 1.,  0.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0.],
        [-1.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  1.],
        [ 0.,  0., -1.,  0.,  0.],
        [ 0.,  0.,  0., -1.,  0.],
        [ 0.,  0.,  0.,  0., -1.]])

In [17]:
corr2d(X, K.t())

tensor([[ 1.,  0.,  0., -1.,  0.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0., -1.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0., -1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.,  0.,  0., -1.,  0.],
        [ 0.,  0.,  0.,  0.,  1.,  0.,  0., -1.]])

In [18]:
X = torch.eye((8))
X

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1.]])

In [19]:
corr2d(X, K)

tensor([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [-1.,  1.,  0.,  0.,  0.,  0.,  0.],
        [ 0., -1.,  1.,  0.,  0.,  0.,  0.],
        [ 0.,  0., -1.,  1.,  0.,  0.,  0.],
        [ 0.,  0.,  0., -1.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  0., -1.,  1.,  0.],
        [ 0.,  0.,  0.,  0.,  0., -1.,  1.],
        [ 0.,  0.,  0.,  0.,  0.,  0., -1.]])

In [20]:
corr2d(X.t(), K)

tensor([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [-1.,  1.,  0.,  0.,  0.,  0.,  0.],
        [ 0., -1.,  1.,  0.,  0.,  0.,  0.],
        [ 0.,  0., -1.,  1.,  0.,  0.,  0.],
        [ 0.,  0.,  0., -1.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  0., -1.,  1.,  0.],
        [ 0.,  0.,  0.,  0.,  0., -1.,  1.],
        [ 0.,  0.,  0.,  0.,  0.,  0., -1.]])

In [21]:
corr2d(X, K.t())

tensor([[ 1., -1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  1., -1.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  1., -1.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  1., -1.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  1., -1.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  1., -1.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  1., -1.]])

2. 在我们创建的Conv2D自动求导时，有什么错误消息？

In [23]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
K = torch.tensor([[1.0, -1.0]])
Y = corr2d(X, K)

conv2d = Conv2D(kernel_size=(1, 2))

lr = 3e-2

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'epoch {i+1}, loss {l.sum():.3f}')

epoch 2, loss 4.970
epoch 4, loss 0.879
epoch 6, loss 0.166
epoch 8, loss 0.035
epoch 10, loss 0.009


In [25]:
conv2d.weight.data

tensor([[ 0.9961, -0.9811]])

In [27]:
conv2d.bias.data

tensor([0.])

3. 如何通过改变输入张量和卷积核张量，将互相关运算表示为矩阵乘法？

In [6]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

In [7]:
def corr2d_v(X, K):  
    h, w = K.shape
    hh, ww = X.shape[0] - h + 1, X.shape[1] - w + 1
    K = K.reshape(-1, 1)
    # Y = torch.zeros((hh, ww, h + w))
    # for i in range(hh):
    #   for j in range(ww):
    #     Y[i, j] = X[i:i+h, j:j+w].reshape(1,-1).squeeze(0)
    # return (Y @ K).squeeze(-1)
    X = torch.stack(([X[i:i+h, j:j+w].reshape(1,-1) for i in range(hh) for j in range(ww)]), 0) 
    Y = X @ K
    return Y.reshape(hh, ww)

In [8]:
corr2d_v(X, K)

tensor([[19., 25.],
        [37., 43.]])